In [10]:
from optimisation_ntn.simulation import Simulation, SimulationConfig
from optimisation_ntn.algorithms.assignment.matrix_based import MatrixBasedAssignment
import numpy as np

sim = Simulation(
    config=SimulationConfig(
        seed=1,
        user_count=10,
        power_strategy="OnDemand",
        save_results=False,
    )
)
assignment_vector = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
matrix_strategy = MatrixBasedAssignment(sim.network)
matrix_strategy.set_assignment_matrix(assignment_vector)
sim.assignment_strategy = matrix_strategy

while sim.current_time < sim.max_time:
    sim.step()

energy_consumed = sim.system_energy_consumed
print(energy_consumed)

1133.3976311574843


In [11]:
sim.reset()
sim.run_with_assignment(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

(1133.3976311574843, 0.5)

In [12]:
from optimisation_ntn.algorithms.assignment.strategy_factory import (
    QLearningAssignment,
    RandomAssignment,
    TimeGreedyAssignment,
    ClosestNodeAssignment,
    HAPSOnlyAssignment,
)

In [13]:
sim.reset()
sim.assignment_strategy = QLearningAssignment(sim.network)
sim.run()

energy_consumed = sim.system_energy_consumed
qos_score = sim.evaluate_qos_satisfaction()
print(energy_consumed)
print(qos_score)

# Load and analyze Q-table
q_learning = QLearningAssignment(sim.network)
q_learning.load_q_table("../qtable.npy")

# Print summary
q_learning.print_q_table_summary()

# Interpret specific state
state = list(q_learning.q_table.keys())[0]  # Get first state
print("\nAnalyzing first state:")
print(q_learning.interpret_state(state))

1851.9985786944908
100.0
Q-table contains 21 states

Best learned action:
State interpretation:

Node Energy Levels:
Node 0: Low
Node 1: Depleted
Node 2: Infinite
Node 3: Depleted
Node 4: Infinite
Node 5: High

Node States:
Node 0: Off
Node 1: Off
Node 2: Off
Node 3: Off
Node 4: Off
Node 5: Off

Request Size: ≤6MB

Best action: Node 4
Q-value: 1.0000

Analyzing first state:
State interpretation:

Node Energy Levels:
Node 0: High
Node 1: Infinite
Node 2: Infinite
Node 3: Infinite
Node 4: Infinite
Node 5: High

Node States:
Node 0: Off
Node 1: Off
Node 2: Off
Node 3: Off
Node 4: Off
Node 5: Off

Request Size: ≤10MB


In [15]:
strategies = [
    QLearningAssignment,
    RandomAssignment,
    TimeGreedyAssignment,
    ClosestNodeAssignment,
    HAPSOnlyAssignment,
]

sim = Simulation(
    config=SimulationConfig(
        seed=1,
        user_count=10,
        power_strategy="OnDemand",
        save_results=False,
    )
)

for strategy in strategies:
    sim.reset()
    strategy_instance = strategy(sim.network)
    sim.assignment_strategy = strategy_instance
    sim.run()
    energy_consumed = sim.system_energy_consumed
    qos_score = sim.evaluate_qos_satisfaction()
    print("-" * 10)
    print(strategy.__name__)
    print(energy_consumed)
    print(qos_score)

----------
QLearningAssignment
1851.9985786944908
100.0
----------
RandomAssignment
2055.7966836204782
100.0
----------
TimeGreedyAssignment
2673.896683620478
90.0
----------
ClosestNodeAssignment
1761.8000000000002
100.0
----------
HAPSOnlyAssignment
1885.0
100.0


In [16]:
# do a brute force search for the best assignment matrix

import itertools
from optimisation_ntn.algorithms.assignment.strategy_factory import (
    AssignmentStrategyFactory,
)
from optimisation_ntn.simulation import Simulation, SimulationConfig
import numpy as np

sim = Simulation(
    config=SimulationConfig(
        seed=1,
        user_count=2,
        power_strategy="OnDemand",
        save_results=False,
    )
)


n_nodes = len(sim.network.compute_nodes)
n_users = len(sim.network.user_nodes)

all_matrices = list(itertools.product(range(n_nodes), repeat=n_users))
print(len(all_matrices))

min_energy = float("inf")
best_matrix = None
# run all the combinations of assignment matrices (n_nodes^n_users)
for i in range(len(all_matrices)):
    assignment_matrix = all_matrices[i]
    sim.reset()
    sim.run_with_assignment(assignment_matrix)
    energy_consumed = sim.system_energy_consumed
    qos_score = sim.evaluate_qos_satisfaction()
    # print(energy_consumed)
    # print(qos_score)
    if qos_score == 100:
        if energy_consumed < min_energy:
            print(f"New best matrix found! Energy: {energy_consumed}, QoS: {qos_score}")
            min_energy = energy_consumed
            best_matrix = assignment_matrix

    if i % 100 == 0:
        print(f"Processed {i/len(all_matrices)*100:.2f}%")

print(best_matrix)
print(min_energy)

36
New best matrix found! Energy: 440.0, QoS: 100.0
Processed 0.00%
New best matrix found! Energy: 424.09952623149684, QoS: 100.0
New best matrix found! Energy: 423.8999999999999, QoS: 100.0
New best matrix found! Energy: 416.19999999999993, QoS: 100.0
New best matrix found! Energy: 400.2995262314967, QoS: 100.0
New best matrix found! Energy: 400.09999999999985, QoS: 100.0
(1, 4)
400.09999999999985
